In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from functools import reduce
import math

# 1. Bernoulli generator

In [ ]:
def bernoulli(s, p, low=0, high=1):
    samples = np.random.choice([low, high], size=s, p=[1-p, p])
    flattened = samples.flatten()
    return samples, np.unique(flattened, return_counts=True)[1] / len(flattened)

n, p = 1_000_000, 0.5
samples, probs = bernoulli(n, p)
bar = plt.bar(np.array([0, 1]), probs)
plt.bar_label(bar, np.bincount(samples.flatten(), minlength=2))
plt.xlabel("k")
plt.ylabel("Discrete probability (P(x = k))")
plt.title("Bernoulli distribution, %s samples and p = %.2f" % (n, p))
plt.show()

In [ ]:
def binomial_empiric(s, n, p):
    res = []
    for _ in range(s):
        res.append(np.where(bernoulli(n, p)[0] == 1)[0].shape[0])
    return res, np.bincount(np.array(res), minlength=n + 1) / s

s, n, p = 100_000, 10, 0.75
plt.figure(figsize=[12, 5])
samples, probs = binomial_empiric(s, n, p)
bar = plt.bar(np.arange(n + 1), probs)
plt.bar_label(bar, np.bincount(samples, minlength=n + 1))
plt.xlabel("k")
plt.ylabel("Discrete probability (P(x = k))")
plt.title("Binomial distribution, %d samples with n = %d and p = %.2f" % (s, n, p))
plt.show()

def binomial_analytic(s, n, p):
    probs = comb(n) * ((p ** np.arange(0, n + 1)) * ((1 - p) ** np.arange(n, -1, -1)))
    return np.random.choice(np.arange(n + 1), size=s, p=probs), probs

def comb(n):
  res = []
  for k in range(n + 1):
    res.append(reduce(lambda x, i: x * (n - i) // (i + 1), range(min(k, n - k)), 1))
  return res
  
plt.figure(figsize=[12, 5])
samples, probs = binomial_analytic(s, n, p)
bar = plt.bar(np.arange(n + 1), probs)
plt.xlabel("k")
plt.ylabel("Discrete probability (P(x = k))")
plt.bar_label(bar, np.bincount(samples, minlength=n + 1))
plt.title("Binomial distribution, %d samples with n = %d and p = %.2f" % (s, n, p))
plt.show()

In [ ]:
def negative_binomial_empiric(s, r, p):
    res = []
    for _ in range(s):
      t, c = 0, 0
      while True:
        if bernoulli(1, p)[0] == 1: c += 1
        else: t += 1
        if c == r: break
      res.append(t)
    maxt = np.max(res)
    return res, np.bincount(np.array(res)) / s, maxt

s, r, p = 10000, 3, 0.5
plt.figure(figsize=[12, 5])
samples, probs, maxt = negative_binomial_empiric(s, r, p)
bar = plt.bar(np.arange(maxt + 1), probs)
plt.bar_label(bar, np.bincount(samples))
plt.xlabel("k")
plt.ylabel("Discrete probability (P(x = k))")
plt.title("Negative binomial distribution, %d samples with r = %d and p = %.2f" % (s, r, p))
plt.show()

In [ ]:
n, p, s, t = 10, 0.7, 5_000, 6
xs = np.array(binomial_empiric(s, n, p)[0])
xs[np.where(xs != t)] = 0
ys = np.cumsum(xs / t) / np.arange(1, len(xs) + 1)
plt.figure(figsize=[12, 5])
plt.title("Plot of P(X = %d), where X ~ Bn(%d, %.2f), estimated empirically over %d samples" % (t, n, p, s))
plt.ylabel("P(x = %d)" % t)
plt.xlabel("samples")
plt.plot(np.arange(1, len(xs) + 1), ys, label='Observed value')
plt.plot(np.arange(1, len(xs) + 1), [sp.special.binom(n, t) * (p ** t) * ((1 - p) ** (n - t))] * len(xs), 'r', label='Theoretical expected value')
plt.legend()
plt.show()

In [ ]:
n, p, s = 10, 0.7, 5000
probs = binomial_analytic(s, n, p)[1]
xs = np.array(binomial_empiric(s, n, p)[0])
cums = []
# I know I am ignoring k=0, but for ease of implementation I am skipping over it, and it is 'good enough' on the graphs
for i in range(1, n + 1):
    xss = np.copy(xs)
    xss[np.where(xss != i)] = 0
    cums.append(np.cumsum(xss / i) / np.arange(1, len(xss) + 1))

ys = np.sum((np.array(cums) - np.tile(probs[1:], (len(xs), 1)).T) ** 2 / len(probs), axis=0)
plt.figure(figsize=[12, 5])
plt.title("Plot of MSE over all probabilities of X ~ Bn(%d, %.2f), estimated empirically over %d samples" % (n, p, s))
plt.ylabel("MSE over all probabilities")
plt.xlabel("samples")
plt.ylim([0, 0.01])
plt.plot(np.arange(1, len(xs) + 1), ys)
plt.show()

In [ ]:
def X(t, N):
  res = bernoulli(np.ceil(t * N).astype(np.int32), 1/2, -1, 1)[0]
  return np.sum(res) / np.sqrt(N)

N = 10000
plt.figure(figsize=[12, 5])
v = []
for t in np.linspace(0, 1, N): v.append(X(t, N))
plt.stem(np.linspace(0, 1, N), v, markerfmt=" ")
plt.xlabel("t = n/N")
plt.ylabel("E[X(n/N)]")
plt.title("Trajectory of process X(n/N) = S_n / sqrt(N), with N = %d and p = 0.5" % N)
plt.show()

In [ ]:
N = 10000
res = bernoulli(N, 1/2, -1, 1)[0]
Sn = np.cumsum(res) / np.sqrt(N)

plt.figure(figsize=[12, 5])
plt.plot(np.linspace(0, 1, N), Sn)
plt.xlabel("t = n/N")
plt.ylabel("E[X(n/N)]")
plt.title("Trajectory of process X(n/N) = S_n / sqrt(N), with N = %d and p = 0.5" % N)
plt.show()

# 2. Symmetry property

In [ ]:
def cantor_empiric(s, low, high, terms=20):
    xs = 2 * bernoulli((s, terms), 1/2, low, high)[0]
    xs = np.sort(np.sum(xs * ((1/3) ** np.arange(1, terms + 1)), axis=1))
    return xs, np.arange(1, len(xs) + 1) / len(xs)

plt.figure(figsize=(14, 5))
plt.title("Cumulative distribution of Cantor distribution with different samples")
for d, c in zip([10, 100, 1000, 10_000, 100_000, 1_000_000], ['g', 'r', 'purple', 'b', 'y']):
    x, y = cantor_empiric(d, 0, 1)
    plt.step(x, y, c, label='%d' % d)
plt.legend()
plt.show()

plt.hist(cantor_empiric(100_000, 0, 1)[0], bins=100)
plt.title('Cantor singular distribution, 100000 samples')
plt.show()

In [ ]:
s = 10_000
fig = plt.figure(figsize=(12, 8))
(ax1, ax2), (ax3, ax4) = fig.subplots(2, 2)
fig.suptitle("Cumulative distributions of Cantor random variable, shown over %d samples" % s)

# Regular cantor
x1, y1 = cantor_empiric(s, 0, 1)
# Flipped over 1/2
x2, y2 = cantor_empiric(s, 0, 1)
x2 = 1 - x2
y2 = 1 - y2
# Only 1/3 
x3, y3 = cantor_empiric(s, 0, 1/3)
# Scale down by 3
x4, y4 = cantor_empiric(s, 0, 1)
x4 /= 3

for ax, x, y, title in zip([ax1, ax2, ax3, ax4], [x1, x2, x3, x4], [y1, y2, y3, y4], \
    ["Regular cantor distribution (Z[0, 1])", "Inverted cantor distribution (1 - Z[0, 1])", "The first third of cantor distr (Z[0, 1/3])", "Scaled down cantor distr (Z[0, 1]/3)"]):
    ax.step(x, y)
    ax.set_title(title)
    ax.set_xlabel("")
    ax.set_ylabel("Cumulative probability")
plt.show()

# 3. From one distribution to another

In [ ]:
def exponential_empiric(s, lambd):
    return -np.log(np.random.uniform(0, 1, s))/lambd
        
s, l, bins = 10_000_000, 0.45, 10000
xrange = np.linspace(0, int(10 * l), bins)
samples = np.bincount(np.digitize(exponential_empiric(s, l), xrange), minlength=bins)[:bins]
plt.figure(figsize=[12, 5])
# Max at x = 0
plt.stem(xrange, samples * l / samples[np.argsort(samples)[-1]], markerfmt=" ")
# Closed form of exponential => λe^(-xλ)
plt.plot(xrange, l * np.exp(- l * xrange), 'r')
plt.xlabel("X")
plt.ylabel("Value of pdf (f(x))")
plt.title("Exponential distribution, %d samples and λ = %.2f" % (s, l))
plt.show()

In [ ]:
def poisson_empiric(s, lambd):
    res = []
    for _ in range(s):
        c, t = -1, 0
        while True:
            c += 1
            # t += exponential_empiric(1, 1/lambd)
            # if t >= lambd ** 2: break
            t += exponential_empiric(1, lambd)
            if t >= 1: break
        res.append(c)
    return res, np.bincount(np.array(res)) / s, np.max(res)

s, l = 100_000, 2.25
samples, probs, maxt = poisson_empiric(s, l)
plt.figure(figsize=[12, 5])
bar = plt.bar(np.arange(maxt + 1), probs)
plt.bar_label(bar, np.bincount(samples))
plt.xlabel("X")
plt.ylabel("Discrete probability (P(x = k))")
plt.title("Poisson distribution, %d samples with λ = %.2f" % (s, l))
plt.show()

def poisson_analytic(s, lambd):
    probs = [lambd ** k * np.exp(-lambd) / math.factorial(k) for k in range(0, int(lambd * 10))]
    probs = np.array(probs) / np.sum(probs)
    res = np.random.choice(np.arange(0, int(lambd * 10)), size=s, p=probs)
    return res, probs, np.max(res)

samples, probs, maxt = poisson_analytic(s, l)
plt.figure(figsize=[12, 5])
bar = plt.bar(np.arange(maxt + 1), probs[:maxt + 1])
plt.bar_label(bar, np.bincount(samples))
plt.xlabel("X")
plt.ylabel("Discrete probability (P(x = k))")
plt.title("Poisson distribution, %d samples with λ = %.2f" % (s, l))
plt.show()

In [ ]:
def normal_empiric(s, mu, sigma):
    r2 = exponential_empiric(s, 1/2)
    phi = np.random.uniform(0, 2 * np.pi, size=s)
    return mu + np.concatenate((np.sqrt(r2) * np.sin(phi), np.sqrt(r2) * np.cos(phi))) * sigma

s, mu, sigma, bins = 1_000_000, -3.5, 2.5, 750
xrange = np.linspace(-5 * sigma, 5 * sigma, bins)
samples = np.bincount(np.digitize(normal_empiric(s, mu, sigma), xrange), minlength=bins)[:bins]
plt.figure(figsize=[12, 5])
# Max at x = μ
plt.stem(xrange, samples * (1/(sigma * np.sqrt(2 * np.pi))) \
    / samples[np.argsort(samples)[-5]], markerfmt=" ")
# Closed form of normal distr e^(-1/2*((x-μ)/σ)^2)/(sqrt(2π)σ)
plt.plot(xrange, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp(-1/2 * ((xrange - mu) / sigma) ** 2), 'r')
plt.xlabel("X")
plt.ylabel("Value of pdf (f(x))")
plt.title("Normal distribution, %d samples, μ = %.2f and σ = %.2f" % (s, mu, sigma))
plt.show()


In [ ]:
def chi_squared_empiric(s, k):
    return np.sum(normal_empiric((s, k), 0, 1) ** 2, axis=-1)

s, k, bins = 2_500_000, 6, 1250
xrange = np.linspace(0, 2 * k, bins)
samples = np.bincount(np.digitize(chi_squared_empiric(s, k), xrange), minlength=bins)[:bins]
plt.figure(figsize=[12, 5])
# Max at x = k - 2
plt.stem(xrange, samples * (k-2) ** (k/2 - 1) * np.exp((2-k)/2) / (2 ** (k/2) * sp.special.gamma(k/2)) \
    / samples[np.argsort(samples)[-5]], markerfmt=" ")
# Closed form of chi squared distribution => x^(k/2 - 1)e^(-x/2)/(2^(k/2)Г(k/2))
plt.plot(xrange, xrange ** (k/2 - 1) * np.exp(-xrange/2) / (2 ** (k/2) * sp.special.gamma(k/2)), 'r')
plt.xlabel("X")
plt.ylabel("Value of pdf (f(x))")
plt.title("Chi squared distribution, %d samples, and %d degrees of freedom" % (s, k))
plt.show()

# 4. LLN and CLT

In [ ]:
a, b, s = 10, 5, 25_000
xs = normal_empiric(s, a, b)

ys = np.cumsum(xs) / np.arange(1, len(xs) + 1)
plt.figure(figsize=[12, 5])
plt.title("Plot of S(n)/n over %d samples" % s)
plt.plot(np.arange(1, len(xs) + 1), ys, label='Observed value')
plt.plot(np.arange(1, len(xs) + 1), [a] * len(xs), 'r', label='Theoretical expected value')
plt.xlabel("samples")
plt.ylabel('S(n)/n')
plt.legend()
plt.show()

print("Final observation: %d" % ys[-1])

In [ ]:
def Z(s, n, mu, sigma):
    return np.sqrt(n) * (np.mean(normal_empiric((s, n), mu, sigma), axis=-1) - mu)# / sigma

a, b, s, n, bins = 7.5, 2.5, 250_000, 250, 500
xrange = np.linspace(-5 * b, 5 * b, bins)
samples = np.bincount(np.digitize(Z(s, n, a, b), xrange), minlength=bins)[:bins]
plt.figure(figsize=[12, 5])
plt.stem(xrange, samples * (1/(b * np.sqrt(2 * np.pi))) / samples[np.argsort(samples)[-5]], markerfmt=" ")
plt.plot(xrange, 1/(b * np.sqrt(2 * np.pi)) * np.exp(-1/2 * (xrange / b) ** 2), 'r', label='normal_distribution_N(0, b^2)')
plt.plot(xrange, 1/(b * np.sqrt(2 * np.pi)) * np.exp(-1/2 * ((xrange - a) / b) ** 2), 'g', label='original_distribution_N(a, b^2)')
plt.xlabel("X")
plt.ylabel("Value of pdf (f(x))")
plt.legend()
plt.title("Plot of resulting distribution, estimated with 10000 iid normal variables and 25000 samples, each with μ = %.2f and σ = %.2f" % (a, b))
plt.show()

In [ ]:
def Z2(s, n, lambd):
    return np.sqrt(n) * (np.mean(exponential_empiric((s, n), lambd), axis=-1) - 1/lambd)

lambd, s, n, bins = 2, 250_000, 10_000, 500
xrange = np.linspace(-5 * b, 5 * b, bins)
samples = np.bincount(np.digitize(Z2(s, n, lambd), xrange), minlength=bins)[:bins]
plt.figure(figsize=[12, 5])
plt.stem(xrange, samples * (1/(b * np.sqrt(2 * np.pi))) / samples[np.argsort(samples)[-5]], markerfmt=" ")
plt.plot(xrange, 1/(b * np.sqrt(2 * np.pi)) * np.exp(-1/2 * (xrange / b) ** 2), 'r', label='standardized_sample_mean')
plt.plot(xrange, 1/(b * np.sqrt(2 * np.pi)) * np.exp(-1/2 * ((xrange - a) / b) ** 2), 'g', label='original distribution')
plt.xlabel("X")
plt.ylabel("Value of pdf (f(x))")
plt.legend()
plt.title("Plot of standardized sample mean, %d samples, estimated with %d iid normal variables, each with μ = %.2f and σ = %.2f" % (s, n, a, b))
plt.show()

# 5. Integral

In [35]:
def I(x):
    res = np.exp(-(np.sum(x, axis=1) + 1 / (2 ** (2 * x.shape[1]) * np.prod(x, axis=1)))) \
        * np.prod(x ** (-np.arange(1, x.shape[1] + 1) / (x.shape[1] + 1)), axis=1)
    res = np.nan_to_num(res, nan=0)
    return res

def monte_carlo_integrate(s, d, step):
    avg, c, total, lambd = [0], 1, 0, 1
    for _ in range(step):
        xs = exponential_empiric((s, d), lambd)
        total = np.sum(I(xs) / np.prod(lambd * np.exp(-xs * lambd), axis=1)) / s
        avg.append(avg[-1] + (total - avg[-1]) / c)
        c += 1
    return avg[1:]

samples_per_step, d, steps = 100_000, 10, 100_000
results = monte_carlo_integrate(samples_per_step, d, steps)
plt.plot(np.arange(0, steps), results)
plt.plot(np.arange(0, steps), [results[-1]] * steps, 'r')
plt.title("Numerical approximation of I(%d), with %d steps, each %d samples\nMonte Carlo Integration" % (d, steps, samples_per_step))
plt.xlabel("Step count")
plt.ylabel("Numerical approximation")
plt.show()
print("Final approximation of integral on %d dimensions, with %d steps, each %d samples: %.10f" % (d, steps, samples_per_step, results[-1]))

In [ ]:
def riemann_sum(xs, w, l, rsum=0):
    ys = I(xs)
    yys = []
    for i, e in enumerate(ys[1:]):
        yys.append([ys[i], e])
    yys = np.array(yys)
    rsum += np.sum(np.max(yys, axis=1)) * w
    return rsum / l

def cartesian(arrays, out=None):
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype
    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)
    m = int(n / arrays[0].size)
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m, 1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m, 1:] = out[0:m, 1:]
    return out

domain, d = np.linspace(0, 5, 10), 1
print("Final approximation of integral on %d dimensions, with %d samples: %.10f" % \
    (d, len(domain), riemann_sum(cartesian(np.tile(domain, (d, 1))), domain[-1] - domain[0], len(domain))))

# 6. Unit simplex

In [ ]:
# Uniform dirichlet -> α = [1](d), turns to a special case where Xs are exponentially distributed
def dirichlet_uniform(s, d):
    xs = exponential_empiric((s, d), 1)
    return xs / np.sum(xs, axis=1, keepdims=True)

s, d, subs = 30_000, 3, [25, 10, 5, 1]
results = dirichlet_uniform(s, d)
fig = plt.figure(figsize=(12, 4))
gs = fig.add_gridspec(2, 2, hspace=0, wspace=0)
(ax1, ax2), (ax3, ax4) = gs.subplots(sharex="col", sharey='row')
fig.suptitle("Flat Dirichlet distribution, (α_n = 1 for all dimensions, or uniform), only first 2 of %d dimensions shown" % d)
for ax, sub in zip([ax1, ax2, ax3, ax4], subs):
    ax.scatter(results[:max(1, s // sub), 0], results[:max(1, s // sub), 1], s=.1)
    ax.set_title("%d samples" % (max(1, s // sub)), y=1.0, pad=-15)
    ax.label_outer()
ax.set_xlabel("X_1")
ax.set_ylabel("X_2")
plt.show()


In [ ]:
def centered_dirichlet(s, d, mu, sigma):
    xs, curr = [], []
    while len(xs) < s:
        curr = normal_empiric((s, d), mu, sigma)
        curr = curr[np.setdiff1d(np.arange(0, s), np.union1d( \
            np.nonzero((curr < 0).sum(axis = 1))[0], \
            np.nonzero((curr > 1).sum(axis = 1))[0]))]
        xs.extend(curr)
    xs = np.array(xs[:s])
    return xs / np.sum(xs, axis=1, keepdims=True)

s, d, mu, sigma, subs = 30_000, 3, 1 / d, 0.4, [25, 10, 5, 1]
results = centered_dirichlet(s, d, mu, sigma)
fig = plt.figure(figsize=(12, 4))
gs = fig.add_gridspec(2, 2, hspace=0, wspace=0)
(ax1, ax2), (ax3, ax4) = gs.subplots(sharex="col", sharey='row')
fig.suptitle("Dirichlet distribution, normally distributed samples, μ = %.2f and σ = %.2f, only first 2 of %d dimensions shown" % (mu, sigma, d))
for ax, sub in zip([ax1, ax2, ax3, ax4], subs):
    ax.scatter(results[:max(1, s // sub), 0], results[:max(1, s // sub), 1], s=.1)
    ax.set_title("%d samples" % (max(1, s // sub)), y=1.0, pad=-15)
    ax.label_outer()
ax.set_xlabel("X_1")
ax.set_ylabel("X_2")
plt.show()

In [ ]:
def general_dirichlet(alpha, s):
    return sp.stats.dirichlet.rvs(alpha, s)

s, d, subs, alpha = 30_000, 3, [25, 10, 5, 1], [2, 2, 2]

results = general_dirichlet(alpha, s)
fig = plt.figure(figsize=(12, 4))
gs = fig.add_gridspec(2, 2, hspace=0, wspace=0)
(ax1, ax2), (ax3, ax4) = gs.subplots(sharex="col", sharey='row')
fig.suptitle("Dirichlet distribution, α = %s, only first 2 of %d dimensions shown" % (str(alpha), d))
for ax, sub in zip([ax1, ax2, ax3, ax4], subs):
    ax.scatter(results[:max(1, s // sub), 0], results[:max(1, s // sub), 1], s=.1)
    ax.set_title("%d samples" % (max(1, s // sub)), y=1.0, pad=-15)
    ax.label_outer()
ax.set_xlabel("X_1")
ax.set_ylabel("X_2")
plt.show()